In [1]:
from pathlib import Path
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
TRAINING_DATA = Path("./data/training_data/")

In [16]:
openai.File.create(
    file = open(TRAINING_DATA/"training_data.jsonl", "rb"),
    purpose = "fine-tune"
)

<File file id=file-L9O7WT2SPWlNiRv8XOaNx2xh at 0x144ece1b0> JSON: {
  "object": "file",
  "id": "file-L9O7WT2SPWlNiRv8XOaNx2xh",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 1506517,
  "created_at": 1696589710,
  "status": "uploaded",
  "status_details": null
}

In [17]:
openai.FineTuningJob.create(training_file = "file-L9O7WT2SPWlNiRv8XOaNx2xh", model = "gpt-3.5-turbo")

<FineTuningJob fine_tuning.job id=ftjob-QOdnk3QxBVDBHM1Lxwk3155U at 0x14344f290> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-QOdnk3QxBVDBHM1Lxwk3155U",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1696589729,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-I4j7CvqT1sUoMLq85ywq27zV",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-L9O7WT2SPWlNiRv8XOaNx2xh",
  "hyperparameters": {
    "n_epochs": "auto"
  },
  "trained_tokens": null,
  "error": null
}

In [28]:
2%2

0

In [26]:
openai.FineTuningJob.retrieve("ftjob-QOdnk3QxBVDBHM1Lxwk3155U")

<FineTuningJob fine_tuning.job id=ftjob-QOdnk3QxBVDBHM1Lxwk3155U at 0x161b32a20> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-QOdnk3QxBVDBHM1Lxwk3155U",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1696589729,
  "finished_at": 1696593019,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::86ddUkes",
  "organization_id": "org-I4j7CvqT1sUoMLq85ywq27zV",
  "result_files": [
    "file-M7IyM6lqF4QZDSXx0RDF9lkz"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-L9O7WT2SPWlNiRv8XOaNx2xh",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 769686,
  "error": null
}

In [1]:
import tokenisation as tkn

/Users/benjamin/Documents/Projects/Code/Python/fed-sentiment/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
await tkn.get_tokenised_press_conference("https://www.federalreserve.gov/mediacenter/files/FOMCpresconf20230503.pdf")

2023-10-08 14:51:30 [info     ] got response                   response_status=200
2023-10-08 14:52:30 [info     ] got response                   response_status=200
2023-10-08 14:55:50 [error    ] got . waiting...               error_count=0 wait_time=4
2023-10-08 15:00:55 [error    ] got . waiting...               error_count=1 wait_time=4
2023-10-08 15:06:00 [error    ] got . waiting...               error_count=2 wait_time=4
2023-10-08 15:08:14 [info     ] got response                   response_status=200


['Good afternoon.',
 'Before discussing today’s meeting, let me comment briefly on recent developments in the banking sector.',
 'Conditions in that sector have broadly improved since early March, and the U.S banking system is sound and resilient.',
 'We will continue to monitor conditions in the sector.',
 'We are committed to learning the right lessons from this episode and will work to prevent events like these from happening again.',
 'As a first step in that process, last week we released Vice Chair for Supervision Barr’s Review of the Federal Reserve’s Supervision and Regulation of Silicon Valley Bank.',
 'The review’s findings underscore the need to address our rules and supervisory practices to make for a stronger and more resilient banking system, and I am confident that we will do so.',
 'From the perspective of monetary policy, our focus remains squarely on our dual mandate to promote maximum employment and stable prices for the American people.',
 'My colleagues and I under